# Inference via REST using Seldon Core

## 3D Image Classification from CT Scans
**Author:** [Bob Kozdemba](https://twitter.com/Bob_Kozdemba)<br>
**Date created:** 2022/08/30<br>
**Description:** Provide slice-level visualization and inference via REST using [Seldon Core](http://seldon.io) on [Openshift](openshift.redhat.com)

*Data preparation and model training credits:*

**Author:** [Hasib Zunair](https://twitter.com/hasibzunair)<br>
**Date created:** 2020/09/23<br>
**Description:** Train a 3D convolutional neural network to predict presence of pneumonia.

## Introduction

This example similates a scenario whereby a radiologist might leverage python-based tools to
assist in the prediction of the presence of viral pneumonia in computer tomography from a collection of (CT) scans. 

### Technologies Used
- Numpy
- Tensorflow
- IpyWidgets
- Requests Library

## Setup

In [1]:
!pip install pip --upgrade -q
!pip install nibabel scipy matplotlib tensorflow ipywidgets -q

In [2]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import logging
import time
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
logging.basicConfig(level=logging.INFO)

In [4]:
#
# Load model from storage.
#
import requests
url = "https://koz.s3.amazonaws.com/models/3d_image_classification.h5"
# url = "https://bob.kozdemba.com/models/3d_image_classification.h5"
model_file = '3d_image_classification.h5'

filename = os.path.join(os.getcwd(), model_file)
keras.utils.get_file(filename, url)

model = keras.models.load_model(filename)

2022-08-30 11:13:43.051657: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
#
# Load volume data from storage.
#
url = "https://koz.s3.amazonaws.com/data/ct-data.zip"
# url = "https://bob.kozdemba.com/data/ct-data.zip"
filename = os.path.join(os.getcwd(), "ct-data.zip")
keras.utils.get_file(filename, url)

# Unzip data in the newly created directory.
with zipfile.ZipFile("ct-data.zip", "r") as z_fp:
    z_fp.extractall("./")

In [6]:
import nibabel as nib
from scipy import ndimage

def read_nifti_file(filepath):
    """Read and load volume"""
    # Read file
    scan = nib.load(filepath)
    # Get raw data
    scan = scan.get_fdata()
    return scan


## Make predictions on a single CT scan

In [7]:
from ipywidgets import interact, interactive, fixed
import matplotlib as mpl
import requests


In [8]:
def predict(filename):
    #
    # payload format
    # payload = {"data": {"ndarray": X.tolist()} }
    #
    
    # 
    # Load the data set for prediction.
    #
    v = read_nifti_file(filename)

    # Local prediction.
    prediction = model.predict(np.expand_dims(v, axis=0))[0]
    logging.info(f'{filename} Local Prediction = {prediction[0]:.3f}')

    #
    # Prediction via REST.
    #
    url = 'http://mymodel-mygroup-ml-mon.apps.ocp.sandbox1911.opentlc.com/api/v1.0/predictions'
    logging.info(f'Serializing and predicting volume {filename} via REST...')
    payload = {"data": {"ndarray": v.tolist()} }
    try:
        t0 = time.time()
        r = requests.post(url, json = payload, timeout = 20)
    except requests.exceptions.ConnectionError:
        logging.info(f'REST connection error!')
        return None
    
    logging.debug(f'response: {r}')
    j = r.json()['data']['ndarray'][0]
    logging.info(f'{filename} REST prediction = {j:.3f}, elapsed time = {time.time() - t0:.1f}s')

    pass

In [9]:
#
# Load a volume so default dimensions are known for interaction widgets.
#
global global_v
study = 0
filename = f'./data/volume{study}.nii.gz'
global_v = read_nifti_file(filename)

In [10]:
#
# Slice along the axial plane.
#
def slice_image(slice = global_v.shape[2] / 2, cmap='none'):
    return mpl.pyplot.imshow(global_v[:, :, slice], cmap=cmap, vmin=global_v.min(), vmax=global_v.max())

In [11]:
def slicer():
    global global_v
    interact(slice_image, slice = (0, global_v.shape[2] - 1, 1), cmap=['gray', 'bone', 'hot', 'magma', 'inferno', 'viridis']);

In [12]:
#
# Slice along the saggital plane.
#
def slice_image2(slice = global_v.shape[1] / 2, cmap='none'):
    return mpl.pyplot.imshow(global_v[:, slice, :].T, cmap=cmap, vmin=global_v.min(), vmax=global_v.max(), origin='lower')

In [13]:
def slicer2():
    global global_v
    interact(slice_image2, slice = (0, global_v.shape[1] - 1, 1), cmap=['gray', 'bone', 'hot', 'magma', 'inferno', 'viridis']);

In [14]:
#
# Slice along the coronal plane.
#
def slice_image3(slice = global_v.shape[0] / 2, cmap='none'):
    return mpl.pyplot.imshow(global_v[slice, :, :].T, cmap=cmap, vmin=global_v.min(), vmax=global_v.max(), origin='lower')

In [15]:
def slicer3():
    global global_v
    interact(slice_image3, slice = (0, global_v.shape[0] - 1, 1), cmap=['gray', 'bone', 'hot', 'magma', 'inferno', 'viridis']);

In [16]:

def set_volume(study = None):
    logging.debug(f'set_volume = {study}')
    if (study != None):
        filename = f'./data/volume{study}.nii.gz'
        logging.debug(f'Loading {filename}')
        filename = f'./data/volume{study}.nii.gz'
        global global_v
        global_v = read_nifti_file(filename)
        logging.debug(f'Calling slicer with {filename}, mean = {global_v.mean()}')
        predict(filename)
        slicer()
        slicer2()
        slicer3()
    else:
        global_v = None
    
    pass



In [17]:
interact(set_volume, study = [0, 1, 2, 3, 4]);

interactive(children=(Dropdown(description='study', options=(0, 1, 2, 3, 4), value=None), Output()), _dom_clas…